In [ ]:
import dask
import dask.array as da
import dask.dataframe as dd
import stackstac
import xarray as xr
import matplotlib.pyplot as plt
from datetime import datetime
from pystac_client import Client

def main(bbox, start_date, end_date):
    # Function to search and retrieve items from a STAC API
    def search_stac_items(bbox, start_date, end_date):
        client = Client.open('https://earth-search.aws.element84.com/v0')  # Example STAC API endpoint
        search = client.search(bbox=bbox, datetime=f"{start_date}/{end_date}", collections=['sentinel-s2-l2a'])
        return search.get_items()
    
    # Function to stack and clip the scenes to the bounding box
    def stack_and_clip(items, assets, bbox):
        return stackstac.stack(items, assets=assets, bounds=bbox)

    # Function to compute the mean NDWI for each scene
    def compute_mean_ndwi(xarray_data):
        # Calculate NDWI
        ndwi = (xarray_data['B3'] - xarray_data['B8']) / (xarray_data['B3'] + xarray_data['B8'])
        mean_ndwi = ndwi.mean(dim=['x', 'y']).compute()  # Compute mean NDWI across x and y dimensions
        return mean_ndwi

    # Function to plot NDWI vs time
    def plot_ndwi(mean_ndwi, timestamps):
        plt.figure(figsize=(12, 6))
        plt.scatter(timestamps, mean_ndwi, label='Mean NDWI', color='blue')
        plt.title('Mean NDWI vs Time')
        plt.xlabel('Time')
        plt.ylabel('Mean NDWI')
        plt.xticks(rotation=45)
        plt.grid()
        plt.legend()
        plt.tight_layout()
        plt.show()

    # Main workflow
    items = search_stac_items(bbox, start_date, end_date)
    
    assets = ['B3', 'B8']  # NDWI requires green (B3) and NIR (B8) bands
    stacked_data = stack_and_clip(items, assets, bbox)

    mean_ndwi = compute_mean_ndwi(stacked_data)

    # Extract timestamps from items
    timestamps = [item.datetime for item in items]
    
    plot_ndwi(mean_ndwi, timestamps)

# Example usage
if __name__ == "__main__":
    bbox = (32.16033157094111, 22.852984440390316, 33.288140399555346, 23.806193394664234)
    start_date = '2017-01-01'
    end_date = '2023-12-31'
    
    main(bbox, start_date, end_date)
